<a href="https://colab.research.google.com/github/niuhaoyu16/tensorflow/blob/main/linear_regression_minibatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [5]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

In [6]:
# create a computation graph
housing = fetch_california_housing()

In [7]:
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name = "y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [8]:
with tf.Session() as sess:
  theta_value = theta.eval()

In [12]:
theta_value
from sklearn.preprocessing import StandardScaler

In [41]:
n_epochs = 1000
learning_rate = 0.01
scaled_housing_data_plus_bias = StandardScaler().fit_transform(housing_data_plus_bias)
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name ="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
# 1 gradients = 2/m * tf.matmul(tf.transpose(X), error)
#2 gradients = tf.gradients(mse, [theta])[0]
#2 training_op = tf.assign(theta, theta - learning_rate*gradients)
# 3 optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
saver = tf.train.Saver()


In [42]:
with tf.Session() as sess:
  sess.run(init)

  for epoch in range(n_epochs):
    if epoch % 100 ==0:
      save_path = saver.save(sess, "/tmp/my_model.ckpt")

    sess.run(training_op)

  best_theta = theta.eval()
  save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

In [43]:
best_theta

array([[ 0.97161865],
       [ 0.8296172 ],
       [ 0.11875131],
       [-0.2655231 ],
       [ 0.30569324],
       [-0.00450308],
       [-0.03932619],
       [-0.89989096],
       [-0.87054604]], dtype=float32)

In [29]:
A = tf.placeholder(tf.float32, shape = (None, 3))
b = A + 5 

In [30]:
with tf.Session() as sess:
  B_val = b.eval(feed_dict={A: [[1,2,3]]})

In [32]:
print(B_val)

[[6. 7. 8.]]


In [34]:
a = np.random.randint(11, size=3)

In [35]:
a

array([9, 4, 0])

In [36]:
X=tf.placeholder(tf.float32, shape=(None, n+1), name = "X")
y = tf.placeholder(tf.float32, shape= (None, 1 ), name = "y")
batch_size = 100
n_batches = int(np.ceil(m/batch_size))
n_epochs = 10

In [38]:
def fetch_batch(epoch, batch_index, batch_size):
  np.random.seed(epoch * n_batches + batch_index)  # 
  indices = np.random.randint(m, size=batch_size)  # 
  X_batch = scaled_housing_data_plus_bias[indices] # 
  y_batch = housing.target.reshape(-1, 1)[indices] # 
  return X_batch, y_batch

In [39]:
with tf.Session() as sess:
  sess.run(init)

  for epoch in range(n_epochs):
    for batch_index in range(n_batches):
      X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
      sess.run(training_op, feed_dict={X:X_batch, y:y_batch})

  best_theta = theta.eval()

In [40]:
best_theta

array([[-0.6319246 ],
       [ 0.8296212 ],
       [ 0.11875211],
       [-0.2655299 ],
       [ 0.3056986 ],
       [-0.00450289],
       [-0.03932635],
       [-0.89988065],
       [-0.87053597]], dtype=float32)